In [ ]:
# import torch
# print(torch.cuda.is_available())
# print(torch.cuda.get_device_name(0))

In [ ]:
# !pip install transformers datasets accelerate wandb

In [1]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


__Get the dataset__

In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# specify gpu for torch
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 287113
    })
    validation: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 13368
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 11490
    })
})

In [ ]:
import re
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\[[^\]]*\]', '', text)
    return text
dataset = dataset.map(lambda x: {'article': clean_text(x['article'])})

Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [ ]:
# Split the dataset into **training** and **validation** sets.

train_test_data = dataset['train'].train_test_split(test_size=0.1)
train_data = train_test_data['train']
val_data = train_test_data['test']


In [ ]:
from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")


# def tokenize_function(examples):
#   return tokenizer(examples["article"], max_length=1024, truncation=True)

#tokenized_data = train_data.map(tokenize_function, batched=True)
# Assuming `tokenizer` is initialized

# max_length=1024
max_length=512

train_data = train_data.map(lambda x: tokenizer(x['article'], padding='max_length', truncation=True, max_length=max_length), batched=True)
val_data = val_data.map(lambda x: tokenizer(x['article'], padding='max_length', truncation=True, max_length=max_length), batched=True)

# Set the labels for the model (if needed, e.g., for summarization)
train_data = train_data.map(lambda x: {'labels': tokenizer(x['highlights'], padding='max_length', truncation=True, max_length=max_length)['input_ids']})
val_data = val_data.map(lambda x: {'labels': tokenizer(x['highlights'], padding='max_length', truncation=True, max_length=max_length)['input_ids']})


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/258401 [00:00<?, ? examples/s]

Map:   0%|          | 0/28712 [00:00<?, ? examples/s]

Map:   0%|          | 0/258401 [00:00<?, ? examples/s]

Map:   0%|          | 0/28712 [00:00<?, ? examples/s]

In [ ]:
train_data

Dataset({
    features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 258401
})

In [ ]:
import pickle

my_object_512 = {'trainData': train_data, 'valData': val_data}

with open('my_object.pkl', 'wb') as f:
    pickle.dump(my_object_512, f)

In [ ]:
#Choose a Pre-trained Model - Use a pre-trained model like `BART`, `T5`, or `Pegasus` for summarization: pretrain on datasets, last layer retrain,

from transformers import AutoModelForSeq2SeqLM

# model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

# smaller model
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")



config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Training the Model: Fine-tune the model using your dataset. Hugging Face’s 'Trainer' API simplifies the process:

from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01
    )

# training_args = TrainingArguments(
#     output_dir="./results/t5-small",  # Save directory
#     evaluation_strategy="epoch",     # Evaluate at each epoch
#     learning_rate=5e-5,              # Adjusted learning rate
#     per_device_train_batch_size=8,   # Batch size per device
#     per_device_eval_batch_size=8,
#     num_train_epochs=3,              # Number of epochs
#     weight_decay=0.01,               # Regularization
#     save_strategy="epoch",           # Save checkpoint at each epoch
#     save_total_limit=2,              # Keep only 2 checkpoints
#     logging_dir="./logs",            # Log directory
#     logging_steps=200,               # Log every 200 steps
# )

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    )

trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss


#### We have done the training and saved it in an other device and thats the reason why we are not able to show the complete trained notebook. Although we have showed in the demo the working model

In [ ]:
## Save model
model.save_pretrained("./T5-ece5831-model")
# Save the tokenizer
tokenizer.save_pretrained('./T5-ece5831-model')